In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
 
import os
import torch
print(os.listdir('../input/snu-2021-1-ds-project-3'))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
training_dir = '../input/snu-2021-1-ds-project-3/train'
test_dir = '../input/snu-2021-1-ds-project-3/test'

In [ ]:
def extract_age(info_file):
    '''
        info file(###.txt)로부터 나이 정보를 뽑아냅니다.
    '''
    with open(info_file, 'r') as f:
        info = f.read()
        for i, line in enumerate(info.split("\n")):
            if line.startswith("#Age"):
                age = float(line.split(": ")[1].strip())
    return age

def extract_sex(info_file):
    '''
        info file(###.txt)로부터 성별 정보를 뽑아냅니다.
    '''
    with open(info_file, 'r') as f:
            info = f.read()
            for i, line in enumerate(info.split("\n")):
                if line.startswith("#Sex"):
                    sex = line.split(": ")[1].strip()
    return sex

def extract_labels(info_file):
    '''
        info file(###.txt)로부터 label(들) 정보를 뽑아냅니다.
    '''
    with open(info_file, 'r') as f:
            info = f.read()
            for i, line in enumerate(info.split("\n")):
                if line.startswith("#Dx"):
                    labels = line.split(": ")[1].strip()
                    labels = labels.split()
    return labels

def read_files(data_directory, is_training=True):
    '''
        data directory(train 또는 test)로부터 모든 sample들의
        id, age, sex, recording, labels 정보를 읽어들여
        (id, age, sex, recording, labels)의 list를 반환합니다.
        is_training=False일 경우엔 labels 정보를 읽어들이지 않습니다.
    '''
    list_id = []
    list_age = []
    list_sex = []
    list_recording = []
    list_labels = []
    for f in os.listdir(data_directory):
        root, extension = os.path.splitext(f)
        if not root.startswith(".") and extension == ".txt":
            list_id.append(int(root))
            info_file = os.path.join(data_directory, root + ".txt")
            recording_file = os.path.join(data_directory, root + ".npy")
            age = extract_age(info_file)
            list_age.append(age)
            sex = extract_sex(info_file)
            list_sex.append(sex)
            with open(recording_file, 'rb') as g:
                recording = np.load(g)
                list_recording.append(recording)
            if is_training:
                labels = extract_labels(info_file)
                list_labels.append(labels)
    if is_training:
        return list(zip(list_id, list_age, list_sex, list_recording, list_labels))
    else:
        return list(zip(list_id, list_age, list_sex, list_recording))

In [ ]:
total_training_set = sorted(read_files(training_dir), key=lambda sample: sample[0])
total_num_training = len(total_training_set)
print(f"Number of total training samples: {total_num_training}")

num_validation = int(total_num_training * 0.2)
num_training = total_num_training - num_validation

print(f'Number of validation samples: {num_validation}')
print(f'Number of training samples: {num_training}')

In [ ]:
# new_features = pd.read_csv('../input/feature-extraction-3/feature_extraction_train_data1.csv')
# new_features.drop(['Unnamed: 0', 'id'] ,axis=1, inplace=True)

# original_features = pd.DataFrame(total_training_set)
# original_features.columns = ['id', 'age', 'sex', 'recording', 'label']

# final_features = pd.concat([original_features, new_features], axis=1)
# final_features = final_features.fillna(0)

In [ ]:
# total_training_set = final_features.values.tolist()

validation_set = total_training_set[:num_validation]
training_set = total_training_set[num_validation:]

In [ ]:
# class Dataset_ECG(torch.utils.data.Dataset):
#     """
#         Build ECG dataset
#     """
#     def __init__(self, dataset, num_classes=12):
#         """
#             dataset을 읽어들여 id, age, sex, recording, labels를 저장한 list를 만들어 줍니다.
#         """
#         self.sample_id = []
#         self.sample_recording = []
#         self.sample_labels = []
#         self.sample_age = []
#         self.sample_sex = []
#         self.sample_f1 = []
#         self.sample_f2 = []
#         self.sample_f3 = []
#         self.sample_f4 = []
#         self.sample_f5 = []
#         self.num_samples = len(dataset)
        
#         for idx in range(self.num_samples):
#             _id, _age, _sex, _recording, _labels, _f1, _f2, _f3, _f4, _f5 = dataset[idx]
#             # model에 input으로 들어가는 data는 torch.Tensor 타입으로 변환해 줍니다.
#             age = torch.tensor(_age)
#             sex = torch.tensor(0) if _sex == "F" else torch.tensor(1)
#             recording = torch.tensor(_recording)
#             labels = torch.tensor(np.zeros(num_classes))
#             for label in _labels:
#                 labels[int(label)] = 1
#             f1 = torch.tensor(_f1)
#             f2 = torch.tensor(_f2)
#             f3 = torch.tensor(_f3)
#             f4 = torch.tensor(_f4)
#             f5 = torch.tensor(_f5)

#             self.sample_id.append(_id)
#             self.sample_age.append(age)
#             self.sample_sex.append(sex)
#             self.sample_recording.append(recording)
#             self.sample_labels.append(labels)
#             self.sample_f1.append(f1)
#             self.sample_f2.append(f2)
#             self.sample_f3.append(f3)
#             self.sample_f4.append(f4)
#             self.sample_f5.append(f5)

#         print(f'Loaded {self.num_samples} samples...')

#     def __len__(self):
#         return self.num_samples

#     def __getitem__(self, idx):
#         return {
#             "id": self.sample_id[idx],
#             "age": self.sample_age[idx],
#             "sex": self.sample_sex[idx],
#             "recording": self.sample_recording[idx],
#             "labels": self.sample_labels[idx],
#             "etc_features": torch.cat((self.sample_age[idx].reshape(1),
#                                        self.sample_sex[idx].reshape(1),
#                                        self.sample_f1[idx].reshape(1),
#                                        self.sample_f2[idx].reshape(1),
#                                        self.sample_f3[idx].reshape(1),
#                                        self.sample_f4[idx].reshape(1),
#                                        self.sample_f5[idx].reshape(1)

#                                       )),
#             "f1": self.sample_f1[idx],
#             "f2": self.sample_f2[idx],
#             "f3": self.sample_f3[idx],
#             "f4": self.sample_f4[idx],
#             "f5": self.sample_f5[idx]
#         }

In [ ]:
class Dataset_ECG(torch.utils.data.Dataset):
    """
        Build ECG dataset
    """
    def __init__(self, dataset, num_classes=12):
        """
            dataset을 읽어들여 id, age, sex, recording, labels를 저장한 list를 만들어 줍니다.
        """
        self.sample_id = []
        self.sample_age = []
        self.sample_sex = []
        self.sample_recording = []
        self.sample_labels = []
        self.num_samples = len(dataset)
        
        for idx in range(self.num_samples):
            _id, _age, _sex, _recording, _labels = dataset[idx]
            # model에 input으로 들어가는 data는 torch.Tensor 타입으로 변환해 줍니다.
            age = torch.tensor(_age)
            sex = torch.tensor(0) if _sex == "F" else torch.tensor(1)
            recording = torch.tensor(_recording)
            labels = torch.tensor(np.zeros(num_classes))
            for label in _labels:
                labels[int(label)] = 1

            self.sample_id.append(_id)
            self.sample_age.append(age)
            self.sample_sex.append(sex)
            self.sample_recording.append(recording)
            self.sample_labels.append(labels)

        print(f'Loaded {self.num_samples} samples...')

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return {
            "id": self.sample_id[idx],
            "age": self.sample_age[idx],
            "sex": self.sample_sex[idx],
            "recording": self.sample_recording[idx],
            "labels": self.sample_labels[idx]
        }

In [ ]:
"""
resnet for 1-d signal data, pytorch version
 
Shenda Hong, Oct 2019
"""
#https://github.com/hsd1503/resnet1d/blob/master/resnet1d.py

import numpy as np
from collections import Counter
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __getitem__(self, index):
        return (torch.tensor(self.data[index], dtype=torch.float), torch.tensor(self.label[index], dtype=torch.long))

    def __len__(self):
        return len(self.data)
    
class MyConv1dPadSame(nn.Module):
    """
    extend nn.Conv1d to support SAME padding
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1):
        super(MyConv1dPadSame, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.conv = torch.nn.Conv1d(
            in_channels=self.in_channels, 
            out_channels=self.out_channels, 
            kernel_size=self.kernel_size, 
            stride=self.stride, 
            groups=self.groups)

    def forward(self, x):
        
        net = x
        
        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)
        
        net = self.conv(net)

        return net

        
class MyMaxPool1dPadSame(nn.Module):
    """
    extend nn.MaxPool1d to support SAME padding
    """
    def __init__(self, kernel_size):
        super(MyMaxPool1dPadSame, self).__init__()
        self.kernel_size = kernel_size
        self.stride = 1
        self.max_pool = torch.nn.MaxPool1d(kernel_size=self.kernel_size)

    def forward(self, x):
        
        net = x
        
        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)
        
        net = self.max_pool(net)
        
        return net
    
class BasicBlock(nn.Module):
    """
    ResNet Basic Block
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups, downsample, use_bn, use_do, is_first_block=False):
        super(BasicBlock, self).__init__()
        
        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.stride = stride
        self.groups = groups
        self.downsample = downsample
        if self.downsample:
            self.stride = stride
        else:
            self.stride = 1
        self.is_first_block = is_first_block
        self.use_bn = use_bn
        self.use_do = use_do

        # the first conv
        self.bn1 = nn.BatchNorm1d(in_channels)
        self.relu1 = nn.ReLU()
        self.do1 = nn.Dropout(p=0.5)
        self.conv1 = MyConv1dPadSame(
            in_channels=in_channels, 
            out_channels=out_channels, 
            kernel_size=kernel_size, 
            stride=self.stride,
            groups=self.groups)

        # the second conv
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.do2 = nn.Dropout(p=0.5)
        self.conv2 = MyConv1dPadSame(
            in_channels=out_channels, 
            out_channels=out_channels, 
            kernel_size=kernel_size, 
            stride=1,
            groups=self.groups)
                
        self.max_pool = MyMaxPool1dPadSame(kernel_size=self.stride)

    def forward(self, x):
        
        identity = x
        
        # the first conv
        out = x
        if not self.is_first_block:
            if self.use_bn:
                out = self.bn1(out)
            out = self.relu1(out)
            if self.use_do:
                out = self.do1(out)
        out = self.conv1(out)
        
        # the second conv
        if self.use_bn:
            out = self.bn2(out)
        out = self.relu2(out)
        if self.use_do:
            out = self.do2(out)
        out = self.conv2(out)
        
        # if downsample, also downsample identity
        if self.downsample:
            identity = self.max_pool(identity)
            
        # if expand channel, also pad zeros to identity
        if self.out_channels != self.in_channels:
            identity = identity.transpose(-1,-2)
            ch1 = (self.out_channels-self.in_channels)//2
            ch2 = self.out_channels-self.in_channels-ch1
            identity = F.pad(identity, (ch1, ch2), "constant", 0)
            identity = identity.transpose(-1,-2)
        
        # shortcut
        out += identity

        return out
    
class ResNet1D(nn.Module):
    """
    
    Input:
        X: (n_samples, n_channel, n_length)
        Y: (n_samples)
        
    Output:
        out: (n_samples)
        
    Pararmetes:
        in_channels: dim of input, the same as n_channel
        base_filters: number of filters in the first several Conv layer, it will double at every 4 layers
        kernel_size: width of kernel
        stride: stride of kernel moving
        groups: set larget to 1 as ResNeXt
        n_block: number of blocks
        n_classes: number of classes
        
    """

    def __init__(self, in_channels, base_filters, kernel_size, stride, groups, n_block, n_classes, downsample_gap=2, increasefilter_gap=4, use_bn=True, use_do=True, verbose=False):
        super(ResNet1D, self).__init__()
        
        self.verbose = verbose
        self.n_block = n_block
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.use_bn = use_bn
        self.use_do = use_do

        self.downsample_gap = downsample_gap # 2 for base model
        self.increasefilter_gap = increasefilter_gap # 4 for base model

        # first block
        self.first_block_conv = MyConv1dPadSame(in_channels=in_channels, out_channels=base_filters, kernel_size=self.kernel_size, stride=1)
        self.first_block_bn = nn.BatchNorm1d(base_filters)
        self.first_block_relu = nn.ReLU()
        out_channels = base_filters
                
        # residual blocks
        self.basicblock_list = nn.ModuleList()
        for i_block in range(self.n_block):
            # is_first_block
            if i_block == 0:
                is_first_block = True
            else:
                is_first_block = False
            # downsample at every self.downsample_gap blocks
            if i_block % self.downsample_gap == 1:
                downsample = True
            else:
                downsample = False
            # in_channels and out_channels
            if is_first_block:
                in_channels = base_filters
                out_channels = in_channels
            else:
                # increase filters at every self.increasefilter_gap blocks
                in_channels = int(base_filters*2**((i_block-1)//self.increasefilter_gap))
                if (i_block % self.increasefilter_gap == 0) and (i_block != 0):
                    out_channels = in_channels * 2
                else:
                    out_channels = in_channels
            
            tmp_block = BasicBlock(
                in_channels=in_channels, 
                out_channels=out_channels, 
                kernel_size=self.kernel_size, 
                stride = self.stride, 
                groups = self.groups, 
                downsample=downsample, 
                use_bn = self.use_bn, 
                use_do = self.use_do, 
                is_first_block=is_first_block)
            self.basicblock_list.append(tmp_block)

        # final prediction
        self.final_bn = nn.BatchNorm1d(out_channels)
        self.final_relu = nn.ReLU(inplace=True)
        # self.do = nn.Dropout(p=0.5)
        self.dense = nn.Linear(out_channels, n_classes)
        # self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        out = x
        
        # first conv
        if self.verbose:
            print('input shape', out.shape)
        out = self.first_block_conv(out)
        if self.verbose:
            print('after first conv', out.shape)
        if self.use_bn:
            out = self.first_block_bn(out)
        out = self.first_block_relu(out)
        
        # residual blocks, every block has two conv
        for i_block in range(self.n_block):
            net = self.basicblock_list[i_block]
            if self.verbose:
                print('i_block: {0}, in_channels: {1}, out_channels: {2}, downsample: {3}'.format(i_block, net.in_channels, net.out_channels, net.downsample))
            out = net(out)
            if self.verbose:
                print(out.shape)

        # final prediction
        if self.use_bn:
            out = self.final_bn(out)
        out = self.final_relu(out)
        out = out.mean(-1)
        if self.verbose:
            print('final pooling', out.shape)
        # out = self.do(out)
        out = self.dense(out)
        if self.verbose:
            print('dense', out.shape)
        # out = self.softmax(out)
        if self.verbose:
            print('softmax', out.shape)
        
        return out    

In [ ]:
    """
    
    Input:
        X: (n_samples, n_channel, n_length)
        Y: (n_samples)
        
    Output:
        out: (n_samples)
        
    Pararmetes:
        in_channels: dim of input, the same as n_channel
        base_filters: number of filters in the first several Conv layer, it will double at every 4 layers
        kernel_size: width of kernel
        stride: stride of kernel moving
        groups: set larget to 1 as ResNeXt
        n_block: number of blocks
        n_classes: number of classes
        
    """

EPOCHS = 30
BATCH_SIZE = 32
LEARNING_RATE = 0.001

kernel_size = 16
stride = 2
n_block = 48
downsample_gap = 6
increasefilter_gap = 12

model = ResNet1D(
    in_channels=2, 
    base_filters=128, # 64 for ResNet1D, 352 for ResNeXt1D
    kernel_size=kernel_size, 
    stride=stride, 
    groups=32, 
    n_block=n_block, 
    n_classes=12, 
    downsample_gap=downsample_gap, 
    increasefilter_gap=increasefilter_gap, 
    use_do=True)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# 위에서 정의한 Dataset_ECG를 활용해 training dataset을 만들어 줍니다.
training_dataset = Dataset_ECG(training_set, num_classes=12)
# Training dataset을 batch 단위로 읽어들일 수 있도록 DataLoader를 만들어줍니다.
training_loader = torch.utils.data.DataLoader(training_dataset, pin_memory=True, batch_size=BATCH_SIZE)

In [ ]:
model.to(device)
model.train()

criterion = torch.nn.BCEWithLogitsLoss() # for multi-label classification
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Training loop
for epoch in range(1, 2):
    print(f'***** Epoch {epoch} *****')
    epoch_training_loss_sum = 0.0
    for i_batch, sample_batched in enumerate(training_loader):
        b_recording = sample_batched["recording"].to(device)
        b_labels = sample_batched["labels"].to(device)
        optimizer.zero_grad()
        b_out = model(b_recording)
        loss = criterion(b_out, b_labels)
        loss.backward()
        optimizer.step()
        epoch_training_loss_sum += loss.item() * b_labels.shape[0]

    epoch_training_loss = epoch_training_loss_sum / num_training
    print(f'training loss of epoch {epoch}: {epoch_training_loss}\n')

In [ ]:
# # Training loop
# for epoch in range(1, EPOCHS+1):
#     LEARNING_RATE=0.0001
#     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
#     print(f'***** Epoch {epoch} *****')
#     epoch_training_loss_sum = 0.0
#     for i_batch, sample_batched in enumerate(training_loader):
#         b_recording = sample_batched["recording"].to(device)
#         b_etc_features = sample_batched["etc_features"].to(device)
#         b_labels = sample_batched["labels"].to(device)
#         optimizer.zero_grad()
#         b_out = model(b_recording, b_etc_features)
#         loss = criterion(b_out, b_labels)
#         loss.backward()
#         optimizer.step()
#         epoch_training_loss_sum += loss.item() * b_labels.shape[0]

#     epoch_training_loss = epoch_training_loss_sum / num_training
#     print(f'training loss of epoch {epoch}: {epoch_training_loss}\n')   
#     model.eval()
#     if epoch > 25:
#         validation_prediction_df = pd.DataFrame(columns=['labels'])
#         validation_prediction_df.index.name = 'id'
#         validation_true_labels_df = pd.DataFrame(columns=['labels'])
#         validation_true_labels_df.index.name = 'id'

#         with torch.no_grad():
#             for idx in range(len(validation_set)):
#                 validation_sample = validation_set[idx]
#                 _, _, _, recording, labels = validation_sample
#                 out = model(torch.tensor(recording).unsqueeze(0).to(device)) # unsqueeze는 batch dimension을 추가해주기 위함
#                 sample_prediction = torch.sigmoid(out).squeeze() > 0.5 # Use 0.5 as a threshold / squeeze는 batch dimension을 제거해주기 위함
#                 indices_of_1s = np.where(sample_prediction.cpu())[0]
#                 str_indices_of_1s = ' '.join(map(str, indices_of_1s))
#                 validation_prediction_df.loc[idx] = [str_indices_of_1s]
        
#                 str_true_labels = ' '.join(labels)
#                 validation_true_labels_df.loc[idx] = [str_true_labels]
#         from sklearn.preprocessing import MultiLabelBinarizer
#         from sklearn.metrics import f1_score

#         mlb = MultiLabelBinarizer(classes=['0','1','2','3','4','5','6','7','8','9','10','11'])
#         mlb.fit(map(str.split, validation_true_labels_df['labels'].values))

#         macro_f1_validation = f1_score(mlb.transform(map(str.split, validation_true_labels_df['labels'].values)), mlb.transform(map(str.split, validation_prediction_df['labels'].values)), average='macro')
#         print(f'macro f1 score on validation set: {macro_f1_validation}')

#         s = '/content/drive/MyDrive/heartprediction/resnet_last1_'
#         y = str(epoch)
#         name = s + y
#         name = name +'.pt'
#         torch.save(model,name)
#     print(time.time()-t0)

In [ ]:
model.eval()

validation_prediction_df = pd.DataFrame(columns=['labels'])
validation_prediction_df.index.name = 'id'
validation_true_labels_df = pd.DataFrame(columns=['labels'])
validation_true_labels_df.index.name = 'id'

with torch.no_grad():
    for idx in range(len(validation_set)):
        validation_sample = validation_set[idx]
        _, _, _, recording, labels = validation_sample
        out = model(torch.tensor(recording).unsqueeze(0).to(device)) # unsqueeze는 batch dimension을 추가해주기 위함
        sample_prediction = torch.sigmoid(out).squeeze() > 0.5 # Use 0.5 as a threshold / squeeze는 batch dimension을 제거해주기 위함
        indices_of_1s = np.where(sample_prediction.cpu())[0]
        str_indices_of_1s = ' '.join(map(str, indices_of_1s))
        validation_prediction_df.loc[idx] = [str_indices_of_1s]
        
        str_true_labels = ' '.join(labels)
        validation_true_labels_df.loc[idx] = [str_true_labels]

In [ ]:
# model.eval()

# validation_prediction_df = pd.DataFrame(columns=['labels'])
# validation_prediction_df.index.name = 'id'
# validation_true_labels_df = pd.DataFrame(columns=['labels'])
# validation_true_labels_df.index.name = 'id'

# with torch.no_grad():
#     for idx in range(len(validation_set)):
#         validation_sample = validation_set[idx]
#         _id, _age, _sex, recording, labels, _f1, _f2, _f3, _f4, _f5= validation_sample
#         _sex = 0 if _sex == "F" else 1
#         etc_features = torch.tensor([_age, _sex, _f1, _f2, _f3, _f4, _f5])
#         #etc_features = torch.tensor([_age, _sex])
#         out = model(torch.tensor(recording).unsqueeze(0).to(device), etc_features.unsqueeze(0).to(device)) # unsqueeze는 batch dimension을 추가해주기 위함
#         sample_prediction = torch.sigmoid(out).squeeze() > 0.5 # Use 0.5 as a threshold / squeeze는 batch dimension을 제거해주기 위함
#         indices_of_1s = np.where(sample_prediction.cpu())[0]
#         str_indices_of_1s = ' '.join(map(str, indices_of_1s))
#         validation_prediction_df.loc[idx] = [str_indices_of_1s]
        
#         str_true_labels = ' '.join(labels)
#         validation_true_labels_df.loc[idx] = [str_true_labels]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

mlb = MultiLabelBinarizer(classes=['0','1','2','3','4','5','6','7','8','9','10','11'])
mlb.fit(map(str.split, validation_true_labels_df['labels'].values))

macro_f1_validation = f1_score(mlb.transform(map(str.split, validation_true_labels_df['labels'].values)), mlb.transform(map(str.split, validation_prediction_df['labels'].values)), average='macro')
print(f'macro f1 score on validation set: {macro_f1_validation}')

In [ ]:
# test_set = sorted(read_files(test_dir, is_training=False), key=lambda sample:sample[0])

# new_features = pd.read_csv('../input/feature-extraction-3/feature_extraction_test_data1.csv')
# new_features.drop(['Unnamed: 0', 'id'], axis=1, inplace=True)

# original_features = pd.DataFrame(test_set)
# original_features.columns = ['id', 'age', 'sex', 'recording']

# final_features = pd.concat([original_features, new_features], axis=1)
# final_features = final_features.fillna(0)
# final_features.to_csv('test_set.csv')
# test_set = final_features.values.tolist()

# num_test = len(test_set)
# print(f'Number of test samples: {num_test}')

In [ ]:
test_set = sorted(read_files(test_dir, is_training=False), key=lambda sample:sample[0])
num_test = len(test_set)
print(f'Number of test samples: {num_test}')

In [ ]:
model.eval()

test_prediction_df = pd.DataFrame(columns=['labels'])
test_prediction_df.index.name = 'id'

with torch.no_grad():
    for idx in range(len(test_set)):
        test_sample = test_set[idx]
        _, _, _, recording = test_sample
        out = model(torch.tensor(recording).unsqueeze(0).to(device)) # unsqueeze는 batch dimension을 추가해주기 위함
        sample_prediction = torch.sigmoid(out).squeeze() > 0.5 # Use 0.5 as a threshold / squeeze는 batch dimension을 제거해주기 위함
        indices_of_1s = np.where(sample_prediction.cpu())[0]
        str_indices_of_1s = ' '.join(map(str, indices_of_1s))
        test_prediction_df.loc[idx] = [str_indices_of_1s]

In [ ]:
test_prediction_df.to_csv('submission3.csv')

In [ ]:
imputed_test_prediction_df = test_prediction_df.replace("", "8")

In [ ]:
imputed_test_prediction_df.to_csv("submission_imputed3.csv")

In [ ]:
# model.eval()

# test_prediction_df = pd.DataFrame(columns=['labels'])
# test_prediction_df.index.name = 'id'

# with torch.no_grad():
#     for idx in range(len(test_set)):
#         test_sample = test_set[idx]
#         _id, _age, _sex, recording, _f1, _f2, _f3, _f4, _f5 = test_sample
#         _sex = 0 if _sex == "F" else 1
#         etc_features = torch.tensor([_age, _sex, _f1, _f2, _f3, _f4, _f5])
#         #etc_features = torch.tensor([_age, _sex])
#         out = model(torch.tensor(recording).unsqueeze(0).to(device), etc_features.unsqueeze(0).to(device)) # unsqueeze는 batch dimension을 추가해주기 위함
#         sample_prediction = torch.sigmoid(out).squeeze() > 0.5 # Use 0.5 as a threshold / squeeze는 batch dimension을 제거해주기 위함
#         indices_of_1s = np.where(sample_prediction.cpu())[0]
#         str_indices_of_1s = ' '.join(map(str, indices_of_1s))
#         test_prediction_df.loc[idx] = [str_indices_of_1s]